# Route Optimization on the Maps

Route optimization on maps involves calculating the most efficient path between multiple points while considering various constraints such as distance, time, or cost. Here, I'll provide a more comprehensive example using Python libraries such as osmnx, networkx, and folium.

We'll focus on the following steps:

Acquire Geospatial Data: Use osmnx to get the street network data for a specified area.
Calculate Optimized Routes: Use networkx to find the shortest path or the most efficient route.
Visualize the Routes: Use folium to create an interactive map with the optimized routes.
Example: Optimizing Routes and Visualizing on a Map

# 1. Setup and Import Libraries
First, ensure you have the necessary libraries installed

In [1]:
!pip install osmnx networkx folium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.


# 2. Acquire Geospatial Data
Download the street network for New York City using osmnx

In [2]:
import osmnx as ox
import networkx as nx
import folium

# Download the street network for New York City
nyc = ox.graph_from_place('New York City, New York, USA', network_type='drive')


# 3. Define Delivery Locations
Specify the delivery locations within New York City:

In [3]:
# Define delivery locations (latitude, longitude)
delivery_locations = {
    'Manhattan': (40.7831, -73.9712),   # Example location in Manhattan
    'Brooklyn': (40.6782, -73.9442),    # Example location in Brooklyn
    'Queens': (40.7282, -73.7949)       # Example location in Queens
    # Add more locations as needed
}


# 4. Calculate Optimized Routes
Calculate the shortest paths between the delivery locations using networkx:

In [4]:
# Find the nearest nodes in the network corresponding to each delivery location
start_nodes = {}
for borough, loc in delivery_locations.items():
    start_nodes[borough] = ox.distance.nearest_nodes(nyc, loc[1], loc[0])

# Calculate the shortest paths between all pairs of delivery locations
optimized_routes = {}
for start_borough, start_node in start_nodes.items():
    routes = {}
    for end_borough, end_node in start_nodes.items():
        if start_borough != end_borough:
            route = nx.shortest_path(nyc, source=start_node, target=end_node, weight='length')
            routes[end_borough] = route
    optimized_routes[start_borough] = routes


# 5. Visualize Routes on a Map
Visualize the optimized routes on an interactive map using folium:

In [5]:
# Create a folium map centered around New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Plot the optimized routes on the map
for start_borough, routes in optimized_routes.items():
    for end_borough, route in routes.items():
        route_coords = []
        for node in route:
            route_coords.append((nyc.nodes[node]['y'], nyc.nodes[node]['x']))
        folium.PolyLine(route_coords, color='blue', weight=5, opacity=0.7).add_to(m)

# Add markers for delivery locations
for borough, loc in delivery_locations.items():
    folium.Marker(location=loc, popup=borough, icon=folium.Icon(color='green')).add_to(m)

# Save and display the map
m.save('nyc_delivery_routes.html')
m
